In [ ]:
from snowflake_dune_tools import get_addres_how_much_funds_transfered

result = get_addres_how_much_funds_transfered.invoke(
    {
        "address": "0xdfd5293d8e347dfe59e90efd55b2956a1343963d",
        "contract_address": "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48",
        "token_sybmol": "USDC",
        "sta_time": "2024-07-29 00:00:00",
        "end_time": "2024-08-06 23:59:59",
    }
)
print(result)

In [ ]:
import snowflake.connector

# Gets the version
ctx = snowflake.connector.connect(
    user='eddie',
    password='Eddiezh1937',
    account='csbrmod-KSB37328',
    warehouse='COMPUTE_WH',
	# authenticator='snowflake',
    # region='AWS_US_WEST_2',
    )
cs = ctx.cursor()
try:
    cs.execute("SELECT current_version()")
    one_row = cs.fetchone()
    print(one_row[0])
finally:
    cs.close()
ctx.close()

In [2]:
from langchain.agents import tool
from snowflake.snowpark import Session
import json

@tool
def get_addres_how_much_funds_transfered(
    address: str,
    contract_address: str,
    token_sybmol: str,
    sta_time: str,
    end_time: str,
) -> str:
    """
    This is useful when you need to get how much and where the token transfered from the address.
    The parameter `address` must be complete a address.
    The parameter `contract_address` must be the erc20 token's complete address.
    The parameters `sta_time` and `end_time` indicate the start time and end time in the format of yyyy-MM-dd HH:mm:ss.
    """
    with open("connections.json") as f:
        connection_parameters = json.load(f)
    session = Session.builder.configs(connection_parameters).create()
    sql = """SELECT
  "from",
  "to",
  sum(value) AS "amount"
FROM
  ETHEREUM_ONCHAIN_DATA.ERC20_ETHEREUM.EVT_TRANSFER transfers
WHERE
  "from" = ?
  AND transfers.contract_address = ?
  AND evt_block_time >= CAST(? AS timestamp)
  AND evt_block_time <= CAST(? AS timestamp)
group by
  "from",
  to
ORDER BY
  amount desc;"""
    results_df = session.sql(
        sql, params=[address, contract_address, sta_time, end_time]
    )
    if not results_df.empty:
        r_list = results_df.apply(
            lambda row: f"Transfer {row['amount']} {token_sybmol} to {row['to']}.",
            axis=1,
        ).tolist()
        session.close()
        return "\n".join(r_list)
    else:
        session.close()
        return "No data found."

result = get_addres_how_much_funds_transfered.invoke(
    {
        "address": "0xdfd5293d8e347dfe59e90efd55b2956a1343963d",
        "contract_address": "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48",
        "token_sybmol": "USDC",
        "sta_time": "2024-07-29 00:00:00",
        "end_time": "2024-08-06 23:59:59",
    }
)
print(result)

SnowparkSQLException: (1304): 01b63957-0002-bd57-007c-1c0700017bce: 001003 (42000): SQL compilation error:
syntax error line 14 at position 2 unexpected 'to'.